# TEST 1

In [20]:
from googlesearch import search

def google_search(query, num_results=10):
    search_results = []
    try:
        for result in search(query, num_results=num_results):
            search_results.append((result.title, result.link))
    except Exception as e:
        print(f"Error: {e}")
    
    return search_results

# Example usage
query = "Python web scraping"
results = google_search(query)
for i, (title, link) in enumerate(results, start=1):
    print(f"{i}. {title}\n{link}\n")


Error: 'str' object has no attribute 'link'


In [35]:
from googlesearch import search


In [36]:
searchres = search("Australia", advanced=True, num_results=10)


In [37]:
for search in searchres:
    print(search)

SearchResult(url=https://australia.gov.au/, title=Official Australian Government information, description=The latest official news, updates and advice from the Australian Government.)
SearchResult(url=https://www.britannica.com/place/Australia, title=Australia | History, Cities, Population, Capital, Map, & Facts, description=4 days ago — Australia, the smallest continent and one of the largest countries on Earth, lying between the Pacific and Indian oceans in the Southern ...)
SearchResult(url=https://www.dfat.gov.au/, title=Homepage | Australian Government Department of Foreign ..., description=The Department of Foreign Affairs and Trade (DFAT) promotes and protects Australia's international interests to support our security and prosperity.)
SearchResult(url=https://www.homeaffairs.gov.au/, title=Department of Home Affairs, description=Home Affairs brings together Australia's federal law enforcement, national and transport security, criminal justice, emergency management, ...)
SearchR

In [15]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.prompts import BaseChatPromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain.memory import ConversationBufferMemory
import re
from langchain.utilities import WikipediaAPIWrapper

wikipedia = WikipediaAPIWrapper()

tools = [
    Tool(
        name="WikipediaSearch",
        func=wikipedia.run,
        description="useful for when you need to answer questions about from wikipedia"
    )
]

template = """Your name is Theodore. As a gentle and kind conversationalist inspired by Bob Ross and Mr. Rogers, please answer the following question:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak kindly and calmly, just like Bob Ross and Mr. Rogers.

Question: {input}
{agent_scratchpad}"""

class CustomPromptTemplate(BaseChatPromptTemplate):
    template: str
    tools: List[Tool]

    def format_messages(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()
llm = ChatOpenAI(temperature=0.5)

llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]

# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history")

agent = initialize_agent(
    tools=tools,
    llm=llm,
    llm_chain=llm_chain,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
    memory=memory,  # Add memory to your agent
)

def converse(user_input: str) -> str:
    # First, add the user input to the conversation memory
    agent.memory.add(HumanMessage(content=user_input))
    
    # Run the agent with the user input and get the response
    agent_response = agent.run(input=user_input, intermediate_steps=[])
    
    # If the agent provided a final answer, add it to the memory and return it
    if "output" in agent_response.return_values:
        output = agent_response.return_values["output"]
        agent.memory.add(agent_response)
        return output

    # If the agent did not provide a final answer, you can handle it accordingly
    return "I am unable to answer your question at this time."

def run_conversation():
    print("Welcome! You can start asking questions. Type 'exit' to end the conversation.")
    
    while True:
        user_input = input("User: ")
        
        # Exit the conversation if the user types 'exit'
        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        # Get the agent's response and print it
        agent_response = converse(user_input)
        print(f"Agent: {agent_response}\n")



In [16]:
run_conversation()

Welcome! You can start asking questions. Type 'exit' to end the conversation.


AttributeError: 'ConversationBufferMemory' object has no attribute 'add'

# Test 2

In [38]:
from langchain.memory import ConversationKGMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.prompts.prompt import PromptTemplate
from langchain.agents import Tool, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.utilities import WikipediaAPIWrapper

wikipedia = WikipediaAPIWrapper()

# Initialize Language Model and Memory
llm = OpenAI(temperature=0)
memory = ConversationKGMemory(llm=llm, return_messages=True)

# Initialize Conversation Knowledge Graph Memory
memory.save_context({"input": "say hi to sam"}, {"ouput": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"ouput": "okay"})

# Initialize Prompt Template
template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)

# Initialize ConversationChain
conversation_with_kg = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=prompt,
    memory=memory
)

# Initialize Conversational Agent
tools = [
    Tool(
        name="Wikipedia Search",
        func=wikipedia.run,
        description="useful for when you need to answer questions that wikipedia may be able to answer"
    ),
]
memory_buffer = ConversationBufferMemory(memory_key="chat_history")
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory_buffer)

def run_conversational_agent(input_text):
    # Update the memory with the new input
    memory.load_memory_variables({"input": input_text})
    # Get the response from the conversational agent
    response = agent_chain.run(input=input_text)
    return response



In [40]:
while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        print("Goodbye!")
        break
    agent_response = run_conversational_agent(user_input)
    print(f"Agent: {agent_response}\n")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Nice to meet you Patrick! What can I do for you today?

> Finished chain.
Agent: Nice to meet you Patrick! What can I do for you today?



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: You are Patrick, the person I just met! Is there anything else I can help you with?

> Finished chain.
Agent: You are Patrick, the person I just met! Is there anything else I can help you with?



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Wikipedia Search
Action Input: Capital of France
Observation: Page: Paris
Summary: Paris (English: ; French pronunciation: ​[paʁi] (listen)) is the capital and most populous city of France, with an official estimated population of 2,102,650 residents as of 1 January 2023 in an area of more than 105 km² (41 sq mi), making it the fourth-most populated city in the European Union as well as the  30th most densely popu

C:\Users\60108555\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\60108555\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Paris
Summary: Paris (English: ; French pronunciation: ​[paʁi] (listen)) is the capital and most populous city of France, with an official estimated population of 2,102,650 residents as of 1 January 2023 in an area of more than 105 km² (41 sq mi), making it the fourth-most populated city in the European Union as well as the  30th most densely populated city in the world in 2022. Since the 17th century, Paris has been one of the world's major centres of finance, diplomacy, commerce, fashion, gastronomy, and science. For its leading role in the arts and sciences, as well as its early and extensive system of street lighting, in the 19th century it became known as "the City of Light". Like London, prior to the Second World War, it was also sometimes called the capital of the world.
The City of Paris is the centre of the Île-de-France region, or Paris Region, with an official estimated population of 12.271.794 habitants on January 1, 2023, or about 19% of the population 